<a href="https://colab.research.google.com/github/manishkakarla/Research_accelerator/blob/master/Custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.vis_utils import plot_model
import math
from tensorflow.keras.layers import Layer
from tensorflow import keras

In [49]:
class IPC(keras.Model):

    def __init__(self, filters,k,alpha):
        '''Initializes the class and sets up the internal variables'''
        # YOUR CODE HERE
        super(IPC,self).__init__()
        #self.inputs=inputs
        #self.input_shap=self.inputs.shape
        self.filters=filters
        self.filters1=int(self.filters*alpha)
        self.filters2=int(self.filters*(1-alpha))
        self.k=k
        self.alpha=alpha
        self.z_i=None
        self.seed=None
        self.activated_beta=None
        self.input_shap=(8,8,512)
    
    #def build(self,inputs):
        '''Create the state of the layer (weights)'''
        # a and b should be initialized with random normal, c (or the bias) with zeros.
        # remember to set these as trainable.
        # YOUR CODE HERE
        self.pointwise1=tf.keras.layers.Conv2D(self.filters2,1,strides=1,use_bias=False)
        
        seed_init=tf.random_uniform_initializer(minval=-1, maxval=0)
        seed_init_val=seed_init(shape=(1,1,self.input_shap[-1],1),dtype='float32')
        #initializer = RandomUniform(-1, 1)
        #config = initializer.get_config()
        #initializer = RandomUniform.from_config(config)
        self.seed=tf.Variable(initial_value=seed_init_val,trainable=True)
        self.z_i=tf.math.multiply(self.seed,self.alpha)

      #print(self.seed)
        #print(self.alpha)
        #self.Z_i=tf.math.multiply(self.seed,self.alpha)
        #print(self.alpha)
       # self.tanh=tf.math.tanh(self.beta)
        
        self.pointwise2=tf.keras.layers.Conv2D(self.filters1,1,strides=1,use_bias=False)
        self.convbeta=tf.keras.layers.Conv2D(self.k,(1,self.filters1+1),strides=1,padding='valid')
        
      
   
    def call(self,inputs):
        '''Defines the computation from inputs to outputs'''
        # YOUR CODE HERE
        #x^2a + xb + c.
        inpu_shape=inputs.shape
        print(self.z_i.shape)
        beta=self.convbeta(self.z_i)
        print(f'beta shs{beta.shape}')
        self.activated_beta=tf.math.tanh(beta)
        pw1=self.pointwise1(inputs)
        print(f"pointwise1{pw1.shape}")
        pw2=self.pointwise2(self.activated_beta)
        print(f"pointwiseshape2{pw2.shape}")
        shape=pw2.shape
        pw2_reshape=tf.reshape(pw2,(1,inpu_shape[1],inpu_shape[2]
                                    ,(shape[1]*shape[2]*shape[3])//(inpu_shape[1]*inpu_shape[2])))
       
        return tf.keras.layers.Concatenate(axis=3)([pw1, pw2_reshape])

In [50]:
input=tf.keras.layers.Input((8,8,512) )

x1 = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(input)
print(x1)
a=IPC(filters=512,alpha=0.875,k=9)
print(f'output{a(x1).shape}')
model= tf.keras.models.Model(input, outputs=a(x1))

KerasTensor(type_spec=TensorSpec(shape=(None, 8, 8, 512), dtype=tf.float32, name=None), name='depthwise_conv2d_21/depthwise:0', description="created by layer 'depthwise_conv2d_21'")
(1, 1, 512, 1)
beta shs(1, 1, 64, 9)
pointwise1(None, 8, 8, 64)
pointwiseshape2(1, 1, 64, 448)
output(1, 8, 8, 512)
(1, 1, 512, 1)
beta shs(1, 1, 64, 9)
pointwise1(None, 8, 8, 64)
pointwiseshape2(1, 1, 64, 448)


In [51]:

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 8, 8, 512)]       0         
                                                                 
 depthwise_conv2d_21 (Depthw  (None, 8, 8, 512)        4608      
 iseConv2D)                                                      
                                                                 
 ipc_23 (IPC)                (1, 8, 8, 512)            41362     
                                                                 
Total params: 45,970
Trainable params: 45,970
Non-trainable params: 0
_________________________________________________________________
